In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh.plotting
import catboost
import sklearn.preprocessing, sklearn.feature_selection, sklearn.model_selection
from sklearn.pipeline import Pipeline, make_pipeline
import sklearn.base
from sklearn.base import BaseEstimator, TransformerMixin
import seaborn as sns
import dateutil.parser
import collections
import sklearn.utils
import itertools
import re
from sklearn.model_selection import train_test_split

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

SEED = 42
np.random.seed = SEED

%matplotlib inline

<IPython.core.display.Javascript object>

In [2]:
from water_transformers import *

In [3]:
y_enc = sklearn.preprocessing.LabelEncoder()
y_enc.fit(pd.Series.from_csv('out_classes.csv'))
y_enc.classes_

array(['functional', 'functional needs repair', 'non functional'], dtype=object)

In [4]:
X_tr, X_cv, X_va, y_tr, y_cv, y_va, X_te = [
    pd.read_csv(fname) for fname in ['X_tr.csv', 'X_cv.csv', 'X_va.csv',
                                     'y_tr.csv', 'y_cv.csv', 'y_va.csv',
                                     'X_te.csv',]
]

datasets = [X_tr, X_cv, X_va, X_te]
for ds in datasets:
    ds.drop(['wpt_name', 'num_private', ], errors='ignore', axis=1, inplace=True)


In [5]:
from sklearn.feature_selection import SelectKBest, chi2

In [6]:
def cat_features_indices(X):
    return np.where(np.array([(X[c].dtype == object)
        or (c.endswith('code'))
        or ('date' in c and '_fq' not in c)
    for c in X.columns.values]))[0]

tr = catboost.Pool(X_tr, y_tr, cat_features=cat_features_indices(X_tr), )

cv = catboost.Pool(X_cv, y_cv, cat_features=cat_features_indices(X_cv))
va = catboost.Pool(X_va, y_va, cat_features=cat_features_indices(X_va))
te = catboost.Pool(X_te, cat_features=cat_features_indices(X_te))

In [ ]:
clf = catboost.CatBoostClassifier(
                           random_seed=SEED,
                           loss_function='MultiClass',
                           verbose=True,
                           calc_feature_importance=True,)
print('create cb instance: %s' % clf)
clf.fit(tr, verbose=True)
print('fit!')

In [ ]:

display('train score: %s' % clf.score(X_tr, y_tr))
display('cv score: %s' % clf.score(X_cv, y_cv))
display('test score: %s' % clf.score(X_te, y_te))
y_test = clf.predict(X_test)
ans=pd.DataFrame({'status_group': y_enc.inverse_transform(
    y_test.astype(int).ravel())},
    index= X_test.iloc[:,0].values)
ans.index.name = 'id'
ans.to_csv(name)
print('saved!')